In [1]:
import numpy as np
import scipy.misc as misc
import os
import glob
import py_img_seg_eval.eval_segm as eval_segm
import imageio

dataset_dir = 'dataset/Cityscapes'

NUM_CLASSES = 4
NUM_EVAL_SAMPLES = 70

FOVmsk = imageio.imread('misc/mask_64.png')
FOVmsk = FOVmsk[:, :, 0]
valid_FOV_index = FOVmsk.reshape(-1) != 0

map_dir = os.path.join(dataset_dir, 'Semantic_Occupancy_Grid_Multi_64', 'val')
pred_folder = 'Semantic_Occupancy_Grid_Multi_64/val/frankfurt'
gt_dir = os.path.join(dataset_dir, 'Cityscapes_frankfurt_GT')
gt_list = sorted(glob.glob(gt_dir+'/*manual.png'))


surfix = ['nn_pred']

for item in surfix:
    pred_list = [dataset_dir+'/'+pred_folder+file_name[-43:-10]+ item + '.png' for file_name in gt_list]
    raw_gt_list = [dataset_dir+'/'+pred_folder+file_name[-43:-11]+ '.png' for file_name in gt_list]


    files = list(zip(gt_list, pred_list, raw_gt_list))
    pixel_accuracy = [[]]
    mean_accuracy = [[]]
    mean_IU = [[]]
    frequency_weighted_IU = [[]]
    a = []
    for current_files in files:

        current_gt = imageio.imread(current_files[0])-1
        current_gt = np.minimum(current_gt, 4)
        current_nn = imageio.imread(current_files[1])
        raw_gt = imageio.imread(current_files[2])

        valid_index_0 = raw_gt.reshape(-1) != 4
        valid_index_1 = current_gt.reshape(-1) != 4
        valid_index  = valid_index_0*valid_index_1*valid_FOV_index

        current_gt = current_gt.reshape(-1)[valid_index]
        current_nn = current_nn.reshape(-1)[valid_index]

        current_gt = current_gt.reshape(1, -1)
        current_nn = current_nn.reshape(1, -1)

        mean_accuracy[0].append(eval_segm.mean_accuracy(current_nn, current_gt))
        mean_IU[0].append(eval_segm.mean_IU(current_nn, current_gt))


    mean_accuracy_avg = np.sum(np.asarray(mean_accuracy), axis = 1) / NUM_EVAL_SAMPLES
    mean_IU_avg = np.sum(np.asarray(mean_IU), axis = 1) / NUM_EVAL_SAMPLES


    print(item, ':')
    print('Mean pixel level accuracy:  %.4f'% mean_accuracy_avg[0])
    print('Mean IU:                    %.4f'% mean_IU_avg[0])

nn_pred :
Mean pixel level accuracy:  0.7577
Mean IU:                    0.6028
